<a href="https://colab.research.google.com/github/abdurrahmanrussel/gmc-product-highlights/blob/main/01_clean_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =====================================================
# ✅ Part 1: Install dependencies and upload products TSV
# =====================================================

# Import libraries
import pandas as pd
from google.colab import files

# Upload products TSV
uploaded = files.upload()
products_file = list(uploaded.keys())[0]
products_df = pd.read_csv(products_file, sep="\t")
print("Loaded products:", products_df.shape)


Saving products_2025-08-30_07_51_53.tsv to products_2025-08-30_07_51_53.tsv
Loaded products: (9889, 41)


In [ ]:
# =====================================================
# ✅ Step 2: Upload Metafields CSV
# =====================================================
from google.colab import files
import pandas as pd

print("Upload Metafields CSV:")
uploaded = files.upload()

metafields_file = None
for fname in uploaded.keys():
    if "metafields" in fname.lower():
        metafields_file = fname
        break

if metafields_file is None:
    raise ValueError("Please upload the metafields CSV file.")

metafields_df = pd.read_csv(metafields_file)
print("Metafields loaded:", metafields_df.shape)

# Preprocess metafields into dict by product id
meta_by_id = {}
id_col = None
for c in metafields_df.columns:
    if 'id' in c.lower() or 'product' in c.lower():
        id_col = c
        break

if id_col:
    for _, r in metafields_df.iterrows():
        meta_by_id[str(r[id_col])] = r.to_dict()

print("Metafields dictionary ready. Total products with metafields:", len(meta_by_id))


# =====================================================
# ✅ Step 3: Upload Example Highlights CSV
# =====================================================
print("\nUpload Example Highlights CSV (optional reference):")
uploaded = files.upload()

example_csv_file = None
for fname in uploaded.keys():
    if "highlights" in fname.lower() or "example" in fname.lower():
        example_csv_file = fname
        break

if example_csv_file is not None:
    example_df = pd.read_csv(example_csv_file)
    print("Example highlights loaded:", example_df.shape)
else:
    print("No example CSV uploaded, continuing without it.")


Upload Metafields CSV:


Saving Product-Metafields-All-Products.csv to Product-Metafields-All-Products.csv
Metafields loaded: (165500, 8)
Metafields dictionary ready. Total products with metafields: 7716

Upload Example Highlights CSV (optional reference):


Saving 11-supplemental_product_highlights_IMPROVED_2025-08-31_092204.csv to 11-supplemental_product_highlights_IMPROVED_2025-08-31_092204.csv
Example highlights loaded: (126, 11)


In [ ]:
# Peek into products file
print("\n=== Products TSV Preview ===")
print(products_df.head(3))
print(products_df.columns.tolist())

# Peek into metafields file
print("\n=== Metafields CSV Preview ===")
print(metafields_df.head(3))
print(metafields_df.columns.tolist())

# Peek into example highlights file (if uploaded)
if example_csv_file is not None:
    print("\n=== Example Highlights Preview ===")
    print(example_df.head(3))
    print(example_df.columns.tolist())



=== Products TSV Preview ===
                                            title  \
0                كوكبة سوار جلد زودياك, برج الدلو   
1              Crossbody kamerataske Litera, Kaki   
2  Horoskooppimerkki rannekoru Stellae, Jousimies   

                                        id       price sale price condition  \
0  shopify_US_6964827127963_40737345863835   34.95 USD        NaN       new   
1  shopify_US_7043384770715_41052252536987  119.99 USD        NaN       new   
2  shopify_US_6964739539099_40736949797019   39.95 USD        NaN       new   

  availability channel feed label language  \
0     in stock  Online         AE       ar   
1     in stock  Online         DK       da   
2     in stock  Online         FI       fi   

                               additional image link  ...  \
0  https://cdn.shopify.com/s/files/1/0495/5363/75...  ...   
1  https://cdn.shopify.com/s/files/1/0495/5363/75...  ...   
2  https://cdn.shopify.com/s/files/1/0495/5363/75...  ...   

          

In [ ]:
import pandas as pd

# =====================================================
# 🔹 Step 1: Prepare metafields lookup by product id
# =====================================================
meta_by_id = {}
for _, row in metafields_df.iterrows():
    pid = str(row["Identification"]).strip().strip("'")
    if pid not in meta_by_id:
        meta_by_id[pid] = {}
    # use Field as key, Translated content if exists, else Default content
    value = row.get("Translated content", None)
    if pd.isna(value) or str(value).strip() == "":
        value = row.get("Default content", None)
    meta_by_id[pid][row["Field"]] = value

# =====================================================
# 🔹 Step 2: Collect product properties
# =====================================================
def collect_properties(row, meta_by_id):
    props = []

    # From products_df
    for col in products_df.columns:
        val = row[col]
        if pd.notna(val) and str(val).strip() != "":
            props.append(f"{col}: {val}")

    # From metafields_df
    pid = str(row["id"]).strip()
    if pid in meta_by_id:
        for k, v in meta_by_id[pid].items():
            if pd.notna(v) and str(v).strip() != "":
                props.append(f"{k}: {v}")

    # Deduplicate while keeping order
    seen = set()
    clean_props = []
    for p in props:
        if p not in seen:
            clean_props.append(p)
            seen.add(p)

    return clean_props

# =====================================================
# 🔹 Step 3: Build highlights dynamically
# =====================================================
highlight_rows = []
max_len = 0

for _, row in products_df.iterrows():
    props = collect_properties(row, meta_by_id)
    entry = {"id": row["id"]}
    for i, prop in enumerate(props, start=1):
        entry[f"product_highlight_{i}"] = prop
    highlight_rows.append(entry)
    max_len = max(max_len, len(props))

# Ensure all rows have the same number of columns
columns = ["id"] + [f"product_highlight_{i}" for i in range(1, max_len+1)]
highlights_df = pd.DataFrame(highlight_rows, columns=columns)

# =====================================================
# 🔹 Step 4: Save to CSV and download
# =====================================================
output_file = "product_highlights_full.csv"
highlights_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print("Saved:", output_file)
print("Max highlights found:", max_len)

from google.colab import files
files.download(output_file)


Saved: product_highlights_full.csv
Max highlights found: 32


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# =====================================================
# 🔹 Step 2b: Define the actual 10 properties to focus on
# =====================================================
# Adjust this list based on what your products actually have
target_properties = [
    "Title", "Brand", "Model", "Color", "Size",
    "Material", "Pattern", "Fits", "Gender", "Age Group"
]

# =====================================================
# 🔹 Step 3c: Collect only these properties
# =====================================================
def collect_actual_properties(row, meta_by_id, target_properties):
    props = []
    pid = str(row["id"]).strip()

    for prop_name in target_properties:
        # Check in products_df first
        if prop_name in row and pd.notna(row[prop_name]) and str(row[prop_name]).strip() != "":
            props.append(f"{prop_name}: {row[prop_name]}")
        # Then check in metafields
        elif pid in meta_by_id and prop_name in meta_by_id[pid]:
            v = meta_by_id[pid][prop_name]
            if pd.notna(v) and str(v).strip() != "":
                props.append(f"{prop_name}: {v}")
        else:
            props.append(None)  # placeholder if missing

    return props

# =====================================================
# 🔹 Step 4: Build highlights using only actual properties
# =====================================================
highlight_rows = []

for _, row in products_df.iterrows():
    props = collect_actual_properties(row, meta_by_id, target_properties)

    entry = {"id": row["id"]}
    for i, prop in enumerate(props, start=1):
        entry[f"product_highlight_{i}"] = prop
    highlight_rows.append(entry)

columns = ["id"] + [f"product_highlight_{i}" for i in range(1, len(target_properties)+1)]
highlights_df = pd.DataFrame(highlight_rows, columns=columns)

# Save
output_file = "product_highlights_actual_10.csv"
highlights_df.to_csv(output_file, index=False, encoding="utf-8-sig")
from google.colab import files
files.download(output_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

# Function to strip HTML tags
def clean_html(text):
    if pd.isna(text):
        return text
    return BeautifulSoup(str(text), "html.parser").get_text(separator=" ", strip=True)

# Apply to all product_highlight columns
highlight_cols = [col for col in products_df.columns if col.startswith("product_highlight_")]
for col in highlight_cols:
    products_df[col] = products_df[col].apply(clean_html)

# Now products_df contains clean text without HTML
products_df.head()


,title,id,price,sale price,condition,availability,channel,feed label,language,additional image link,...,product type,purchasable without shipping,sale price effective date,sell on google quantity,shipping weight,size,size system,size type,sku,update type
0,"كوكبة سوار جلد زودياك, برج الدلو",shopify_US_6964827127963_40737345863835,34.95 USD,NaN,new,in stock,Online,AE,ar,https://cdn.shopify.com/s/files/1/0495/5363/75...,...,Apparel & Accessories > Jewelry > Bracelets,NaN,NaN,NaN,50 g,NaN,NaN,NaN,NaN,NaN
1,"Crossbody kamerataske Litera, Kaki",shopify_US_7043384770715_41052252536987,119.99 USD,NaN,new,in stock,Online,DK,da,https://cdn.shopify.com/s/files/1/0495/5363/75...,...,Luggage & Bags > Messenger Bags,NaN,NaN,NaN,1120 g,27cm x 35cm x 11cm,NaN,NaN,NaN,NaN
2,"Horoskooppimerkki rannekoru Stellae, Jousimies",shopify_US_6964739539099_40736949797019,39.95 USD,NaN,new,in stock,Online,FI,fi,https://cdn.shopify.com/s/files/1/0495/5363/75...,...,Apparel & Accessories > Jewelry > Bracelets,NaN,NaN,NaN,13 g,NaN,NaN,NaN,NaN,NaN
3,"Läder iPhonefodral Croco Glossy, iPhone 14 / G...",shopify_US_7663237366017_43531638767873,74.50 USD,NaN,new,in stock,Online,SE,sv,https://cdn.shopify.com/s/files/1/0495/5363/75...,...,Electronics > Communications > Telephony > Mob...,NaN,NaN,NaN,220 g,NaN,NaN,NaN,NaN,NaN
4,Alcantara iPhone Case iPhone 12 / Blue,shopify_US_5741825032347_36530108399771,69.99 USD,NaN,new,out of stock,Online,US,en,https://cdn.shopify.com/s/files/1/0495/5363/75...,...,Electronics > Communications > Telephony > Mob...,NaN,NaN,0.0,0 g,NaN,NaN,NaN,36965924-for-iphone-12-blue,NaN


In [ ]:
import pandas as pd
import re
from difflib import SequenceMatcher

# -----------------------------
# Step 1: Define property mapping and templates
# -----------------------------
property_mapping = {
    "title": ["title", "name", "product_name"],
    "material": ["material", "main_material", "composition"],
    "color": ["color", "shade", "colour"],
    "pattern": ["pattern", "texture", "finish"],
    "fit": ["fit", "size_fit", "compatible_with"],
    "weight": ["weight", "product_weight"],
    "compatibility": ["compatibility", "device_compatibility"],
    "protection": ["protection", "features_protection"],
    "special_features": ["features", "special_features", "highlights"],
    "description": ["description", "long_description", "details"]
}

highlight_order = [
    "title", "material", "pattern", "color", "fit",
    "weight", "compatibility", "protection", "special_features", "description"
]

templates = {
    "fit": lambda val: f"Fits {val}" if val else None,
    "weight": lambda val: f"Approx. {val} weight" if val else None,
    "protection": lambda val: f"{val}" if val else None,
    "special_features": lambda val: f"{val}" if val else None,
    "compatibility": lambda val: f"Compatible with {val}" if val else None,
    "material": lambda val: f"Made from {val}" if val else None,
    "pattern": lambda val: f"{val} texture" if val else None,
    "title": lambda val: f"{val}" if val else None,
    "color": lambda val: f"Color: {val}" if val else None
}

# -----------------------------
# Step 2: Build metafield lookup
# -----------------------------
meta_by_id = {}
for _, row in metafields_df.iterrows():
    pid = str(row["Identification"]).strip().strip("'")
    if pid not in meta_by_id:
        meta_by_id[pid] = {}
    value = row.get("Translated content") or row.get("Default content")
    if pd.notna(value) and str(value).strip():
        meta_by_id[pid][row["Field"]] = str(value).strip()

# -----------------------------
# Step 3: Parse description into short phrases (LIMITED)
# -----------------------------
def parse_description(text, max_sentences=5):
    if not text or str(text).strip() == "":
        return []
    sentences = re.split(r'[.;,]\s*', text)
    # Only take the first max_sentences
    return [s.strip() for s in sentences if len(s.strip()) > 3][:max_sentences]

# -----------------------------
# Step 3b: Truncate text
# -----------------------------
def truncate_text(text, max_len=200):
    if not text:
        return None
    return text if len(text) <= max_len else text[:max_len].rsplit(' ', 1)[0] + "..."

# -----------------------------
# Step 3c: Similarity check for deduplication
# -----------------------------
def is_similar(a, b, threshold=0.8):
    return SequenceMatcher(None, a, b).ratio() > threshold

# -----------------------------
# Step 4: Generate highlights
# -----------------------------
def generate_highlights(row, meta_by_id):
    pid = str(row["id"]).strip()
    highlights = []

    for prop in highlight_order:
        found = False
        # Check product columns
        for col in property_mapping[prop]:
            if col in row and pd.notna(row[col]) and str(row[col]).strip():
                val = str(row[col]).strip()
                if prop == "description":
                    for desc in parse_description(val):
                        highlights.append(truncate_text(desc))
                else:
                    highlights.append(truncate_text(templates[prop](val)))
                found = True
                break
        # Check metafields if not found
        if not found and pid in meta_by_id:
            for key in property_mapping[prop]:
                if key in meta_by_id[pid] and meta_by_id[pid][key]:
                    val = meta_by_id[pid][key]
                    if prop == "description":
                        for desc in parse_description(val):
                            highlights.append(truncate_text(desc))
                    else:
                        highlights.append(truncate_text(templates[prop](val)))
                    found = True
                    break

    # Deduplicate (remove similar entries)
    clean_highlights = []
    for h in highlights:
        if h and not any(is_similar(h, ch) for ch in clean_highlights):
            clean_highlights.append(h)

    # Fill up to 10 highlights
    while len(clean_highlights) < 10:
        clean_highlights.append("N/A")

    return clean_highlights[:10]

# -----------------------------
# Step 5: Build DataFrame
# -----------------------------
highlight_rows = []
for _, row in products_df.iterrows():
    highlights = generate_highlights(row, meta_by_id)
    entry = {"id": row["id"]}
    for i, h in enumerate(highlights, 1):
        entry[f"product_highlight_{i}"] = h
    highlight_rows.append(entry)

highlights_df = pd.DataFrame(highlight_rows)

# -----------------------------
# Step 6: Save CSV
# -----------------------------
output_file = "product_highlights_consumer_friendly.csv"
highlights_df.to_csv(output_file, index=False, encoding="utf-8-sig")

from google.colab import files
files.download(output_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import re
from difflib import SequenceMatcher
import pandas as pd

# -----------------------------
# Property mapping and templates
# -----------------------------
property_mapping = {
    "title": ["title", "name", "product_name"],
    "material": ["material", "main_material", "composition"],
    "color": ["color", "shade", "colour"],
    "pattern": ["pattern", "texture", "finish"],
    "fit": ["fit", "size_fit", "compatible_with"],
    "weight": ["weight", "product_weight"],
    "compatibility": ["compatibility", "device_compatibility"],
    "protection": ["protection", "features_protection"],
    "special_features": ["features", "special_features", "highlights"],
    "description": ["description", "long_description", "details"]
}

highlight_order = [
    "title", "material", "pattern", "color", "fit",
    "weight", "compatibility", "protection", "special_features", "description"
]

templates = {
    "fit": lambda val: f"Fits {val}" if val else None,
    "weight": lambda val: f"Approx. {val} weight" if val else None,
    "protection": lambda val: val if val else None,
    "special_features": lambda val: val if val else None,
    "compatibility": lambda val: f"Compatible with {val}" if val else None,
    "material": lambda val: f"Made from {val}" if val else None,
    "pattern": lambda val: f"{val} texture" if val else None,
    "title": lambda val: val if val else None,
    "color": lambda val: f"Color: {val}" if val else None
}

# -----------------------------
# HTML cleaning
# -----------------------------
def clean_html(text):
    if not text: return ""
    text = re.sub(r'<[^>]+>', '', str(text))  # remove tags
    text = re.sub(r'\s+', ' ', text)  # normalize whitespace
    return text.strip()

# -----------------------------
# Metafield lookup
# -----------------------------
meta_by_id = {}
for _, row in metafields_df.iterrows():
    pid = str(row["Identification"]).strip().strip("'")
    if pid not in meta_by_id:
        meta_by_id[pid] = {}
    value = row.get("Translated content") or row.get("Default content")
    if pd.notna(value) and str(value).strip():
        meta_by_id[pid][row["Field"]] = str(value).strip()

# -----------------------------
# Helpers
# -----------------------------
def parse_description(text, max_sentences=5):
    """
    Split text into full sentences using punctuation and keep only the first max_sentences.
    """
    text = clean_html(text)
    if not text: return []

    # Split by sentence-ending punctuation (. ! ?)
    sentences = re.split(r'(?<=[.!?])\s+', text)

    # Keep sentences with enough length
    sentences = [s.strip() for s in sentences if len(s.strip()) > 3]

    return sentences[:max_sentences]

def truncate_text(text, max_len=200):
    if not text: return None
    return text if len(text) <= max_len else text[:max_len].rsplit(' ', 1)[0] + "..."

def is_similar(a, b, threshold=0.8):
    return SequenceMatcher(None, a, b).ratio() > threshold

# -----------------------------
# Generate highlights
# -----------------------------
def generate_highlights(row, meta_by_id):
    pid = str(row["id"]).strip()
    highlights = []

    for prop in highlight_order:
        found = False
        for col in property_mapping[prop]:
            if col in row and pd.notna(row[col]) and str(row[col]).strip():
                val = clean_html(row[col])
                if prop == "description":
                    for desc in parse_description(val):
                        highlights.append(truncate_text(desc))
                else:
                    highlights.append(truncate_text(templates[prop](val)))
                found = True
                break
        if not found and pid in meta_by_id:
            for key in property_mapping[prop]:
                if key in meta_by_id[pid] and meta_by_id[pid][key]:
                    val = clean_html(meta_by_id[pid][key])
                    if prop == "description":
                        for desc in parse_description(val):
                            highlights.append(truncate_text(desc))
                    else:
                        highlights.append(truncate_text(templates[prop](val)))
                    break

    # Deduplicate
    clean_highlights = []
    for h in highlights:
        if h and not any(is_similar(h, ch) for ch in clean_highlights):
            clean_highlights.append(h)

    # Fill up to 10
    while len(clean_highlights) < 10:
        clean_highlights.append("N/A")

    return clean_highlights[:10]

# -----------------------------
# Build final DataFrame
# -----------------------------
highlight_rows = []
for _, row in products_df.iterrows():
    highlights = generate_highlights(row, meta_by_id)
    entry = {"id": row["id"]}
    for i, h in enumerate(highlights, 1):
        entry[f"product_highlight_{i}"] = h
    highlight_rows.append(entry)

highlights_df = pd.DataFrame(highlight_rows)

# Save and download
output_file = "product_highlights_consumer_friendly.csv"
highlights_df.to_csv(output_file, index=False, encoding="utf-8-sig")

from google.colab import files
files.download(output_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>